In [1]:
!pip install pretrainedmodels

     |████████████████████████████████| 58 kB 822 kB/s eta 0:00:01
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=39a8a72f8ea3ac8626e4b350c38b5283d4f60f16c4248a645f02e8f4438d7dea
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pretrainedmodels
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pair_list_test = [(57583, 82), (57586, 4152), (57680, 2206)]

In [4]:
import torch.nn.functional as F

In [5]:
import os
import cv2
from tqdm import tqdm 
from PIL import Image
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset

import torch
import torchvision.models as models

import albumentations as A
# from albumentations.pytorch.transforms import ToTensor, ToTensorV2
# from albumentations.augmentations.transforms import Normalize

from albumentations import Compose, RandomCrop, Normalize, HorizontalFlip, Resize
from albumentations.pytorch import ToTensor

In [6]:
! ls ../input/resnet18-on-bboxes

__notebook__.ipynb  big_helmet_test.csv   helmet_crop_train
__output__.json     big_helmet_train.csv  well_test_dataset.csv
__results__.html    custom.css		  well_train_dataset.csv
__results___files   helmet_crop_test


In [8]:
def convert_name(row):
#     print(row)
    input_name = row.image_name
    playerLabel = row.label
    impact = row.impact
    return f"{input_name[:-4]}_{playerLabel}_{int(impact)}.png"


In [9]:
train_df = pd.read_csv('../input/resnet18-on-bboxes/well_train_dataset.csv')
test_df = pd.read_csv('../input/resnet18-on-bboxes/well_test_dataset.csv')

train_df['image_name'] = train_df.apply(lambda x: convert_name(x), axis=1)
test_df['image_name'] = test_df.apply(lambda x: convert_name(x), axis=1)

train_df.to_csv('well_train_dataset.csv')
test_df.to_csv('well_test_dataset.csv')



In [10]:
print(len(train_df),len(test_df))
train_df.head()

55095 12660


,gameKey,playID,view,video,frame,label,impact,impactType,confidence,visibility,image_name,x,y,w,h
0,57584,336,Endzone,57584_000336_Endzone.mp4,29,V59,0.0,0,0.0,0.0,57584_000336_Endzone_029_V59_0.png,661,143,22,36
1,57584,336,Endzone,57584_000336_Endzone.mp4,29,V54,0.0,0,0.0,0.0,57584_000336_Endzone_029_V54_0.png,378,146,22,34
2,57584,336,Endzone,57584_000336_Endzone.mp4,29,V97,0.0,0,0.0,0.0,57584_000336_Endzone_029_V97_0.png,245,243,24,32
3,57584,336,Endzone,57584_000336_Endzone.mp4,29,V95,1.0,0,0.0,0.0,57584_000336_Endzone_029_V95_1.png,789,242,24,35
4,57584,336,Endzone,57584_000336_Endzone.mp4,29,H51,0.0,0,0.0,0.0,57584_000336_Endzone_029_H51_0.png,612,227,30,24


In [11]:
class ImageDatasetBoxes(Dataset) :
    def __init__(self, df=None, transform=None, img_size=(100,100), mask= False, input_folder="../input/resnet18-on-bboxes/", name="train") :
        self.df = df
        self.img_size = img_size
        self.transform = transform
        self.mask = mask
        
#         print(f"name={name}")
        
        if name == 'train':
            self.input_folder = os.path.join(input_folder,'helmet_crop_train')
        elif name == 'test':
            self.input_folder = os.path.join(input_folder,'helmet_crop_test')
            
    def __len__(self) :
        return len(self.df)
    
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_path = os.path.join(self.input_folder, row.image_name)
        image = cv2.imread(img_path)
    
        if self.transform:
            transformed_img = self.transform(image=image)['image']
        
        if int(row.impact) == 0:
            label = torch.tensor([1.0,0.0])
        elif int(row.impact) == 1:
            label = torch.tensor([0.0,1.0])
        else:
            print(row.impact)
            print(type(row.impact))
            assert False, 'incorrect label type'
        return transformed_img, label
    

In [12]:
transform = A.Compose(
    [
        Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        ),
        ToTensor()
    ]
)

train_dataset = ImageDatasetBoxes(train_df,transform= transform )
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = ImageDatasetBoxes(test_df,transform= transform, name = "test")
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


# Training scrip

In [13]:
# epoch_num = 7

train_losses = []
test_losses = []
# criterion = torch.nn.CrossEntropyLoss()




In [14]:
def check_diff_net(model, model2):
    for name, param in model.named_parameters():
        for name2, param2 in model2.named_parameters():
        #case load weight and non load weight
            s = torch.tensor(0)
            if(name==name2):
#                 print(name)
                temp = (param - param2).sum()
                s = s + temp 
#                 print(temp)
            break
        break
    
    if(s == torch.tensor(0)):
        print('same net')
    else:
        print('different net')
        
# check_diff_net(resnet18, t['model'])

In [15]:
from sklearn.metrics import classification_report,confusion_matrix


In [16]:
def run_from_epoch(model, start_epoch, epoch_num, model_name): 
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    train_losses = []
    test_losses = []
#     criterion = torch.nn.CrossEntropyLoss()
    criterion = torch.nn.BCEWithLogitsLoss(weight = torch.tensor([1,5]).to(device))
    
#     assert start_epoch < epoch_num, 'start_epoch > epoch_num'
    for e in range(start_epoch,epoch_num):
        
        sum_train = 0
        sum_test = 0
        train_count = 0
        test_count = 0
        for b, l in tqdm(train_loader) :
            b = b.to(device)
            l = l.to(device)

            model.train()                        
            # zero the parameter gradients
            optimizer.zero_grad()


            out = model(b)

            labels = l.argmax(1)
#             train_loss = criterion(out, labels)
            train_loss = criterion(out, l)
            train_loss.backward()
            optimizer.step()

            with torch.set_grad_enabled(False):
                sum_train += train_loss.to('cpu').item()
                train_count += 1

        model.eval()
        with torch.no_grad():            
            gts = []
            preds = []
    #     with torch.set_grad_enabled(False):
            for b, l in tqdm(test_loader) :
                b = b.to(device)
                l = l.to(device)

                out = model(b)
                labels = l.argmax(1)
                
                out_ = out.argmax(1)
                preds.append(out_.to('cpu').numpy())
        
                labels = l.argmax(1)
                gts.append(labels.to('cpu').numpy())

                test_loss = criterion(out, l)

                sum_test += test_loss.to('cpu').item()
                test_count += 1
                
            gts = np.concatenate(gts) 
            preds = np.concatenate(preds)
            dic = {
                'model': model,
                'test_loss': test_loss.item(),
                'training_loss': train_loss.item(),
                'gts': gts,
                'preds': preds,
            }
            print(classification_report(gts, preds))
            print(confusion_matrix(gts, preds))
            print(sum_train/train_count)
            print(sum_test/test_count)
            torch.save(dic, f"{model_name}_epoch_no_mask{e}.pt")
            
            train_losses.append(sum_train/train_count)
            test_losses.append(sum_test/test_count)
    return train_losses, test_losses

In [18]:
# model = torch.load('resnet_epoch_no_mask0.pt')['model']
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model_name = 'se_resnext50_32x4d' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model.avg_pool = torch.nn.AdaptiveAvgPool2d(1)
model.last_linear = torch.nn.Linear(
    model.last_linear.in_features,
    2,
)
model = model.to(device)
train_losses, test_losses = run_from_epoch(model, 0, 4, model_name)

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth" to /root/.cache/torch/hub/checkpoints/se_resnext50_32x4d-a260b3a4.pth


NameError: name 'device' is not defined

In [ ]:
# test_losses = []
# train_losses = []
# for i in range(7):
#     state =  torch.load(f"./resnet_epoch_no_mask{i}.pt")
#     test_losses.append(state['test_loss'])
#     train_losses.append(state['training_loss'])
#     del state
    

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(epoch), train_losses, 'b')
plt.plot(range(epoch), test_losses, 'r')
plt.legend(['Train Losses','Test Losses'])
plt.show()

In [ ]:
# print('tét')

In [ ]:
np.random.rand(1,2)[0]

In [ ]:
# model = torch.load('resnet_epoch_no_mask4.pt')['model']

# model.eval()
# with torch.no_grad():            
# #     with torch.set_grad_enabled(False):
#     for b, l in tqdm(test_loader) :
#         b = b.to(device)
#         l = l.to(device)

#         out = model(b)
#         out_ = out.argmax(1)
#         predicts.append(out_.to('cpu').numpy())
        
#         labels = l.argmax(1)
#         gt.append(labels.to('cpu').numpy())
#         test_loss = criterion(out, labels)

#         sum_test += test_loss.to('cpu').item()
        
#         dic = {
#             'model': model,
#             'test_loss': test_loss.item(),
#             'training_loss': train_loss.item()
#         }
        

In [ ]:
# np.array(predicts)

In [ ]:
# print(classification_report(np.concatenate(gt), np.concatenate(predicts)))